## Create total acres grouped by day (final_daily_total_plus_wind)
### Create a dataframe with a single row per pay, containing a column with total acres, columns for wind data, and column for aqi.  Write to a table named final_daily_total_plus_wind


In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

%matplotlib inline


In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
data = pd.read_sql("select * from binned_by_acres", con=engine)
data.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,index,date,overall_aqi,fire_id,size,distance,bins
0,0,2001-01-01,153,8659.0,0.1,380.928215,bigger then 300
1,1,2001-01-01,153,9387.0,0.1,136.804378,within 150
2,2,2001-01-01,153,17163.0,0.1,251.976491,within 300
3,3,2001-01-01,153,30114.0,0.1,200.715760,within 250
4,4,2001-01-01,153,30201.0,0.1,467.441792,bigger then 300


In [4]:
data = data[['date', 'overall_aqi', 'fire_id', 'size']]
data.head()

,date,overall_aqi,fire_id,size
0,2001-01-01,153,8659.0,0.1
1,2001-01-01,153,9387.0,0.1
2,2001-01-01,153,17163.0,0.1
3,2001-01-01,153,30114.0,0.1
4,2001-01-01,153,30201.0,0.1


## Group by date (ie create total acres  per day)
* Note: because we only count acres burned on its end date, grouping by day is likely to be oversimplistic

In [5]:
grouped_acres = data.groupby([pd.Grouper(key="date", freq="D")])['size'].sum()

In [6]:
type(grouped_acres)

pandas.core.series.Series

In [8]:
grouped_acres.head()

date
2001-01-01      1.5
2001-01-02      0.4
2001-01-03      0.5
2001-01-04      9.8
2001-01-05    146.0
Freq: D, Name: size, dtype: float64

In [9]:
grouped_acres = grouped_acres.to_frame()

In [10]:
grouped_acres.head()

,size
date,
2001-01-01,1.5
2001-01-02,0.4
2001-01-03,0.5
2001-01-04,9.8
2001-01-05,146.0


In [11]:
grouped_acres = grouped_acres.fillna(0)

In [12]:
grouped_acres.head()

,size
date,
2001-01-01,1.5
2001-01-02,0.4
2001-01-03,0.5
2001-01-04,9.8
2001-01-05,146.0


## Create dataframe with AQI

In [41]:
aqi = data.groupby([pd.Grouper(key="date", freq="D")])['overall_aqi'].mean()
aqi.head()

date
2001-01-01    153.0
2001-01-02    140.0
2001-01-03    140.0
2001-01-04    107.0
2001-01-05    124.0
Freq: D, Name: overall_aqi, dtype: float64

In [42]:
aqi = aqi.to_frame()

In [43]:
aqi.head()

,overall_aqi
date,
2001-01-01,153.0
2001-01-02,140.0
2001-01-03,140.0
2001-01-04,107.0
2001-01-05,124.0


## Create dataframe with wind data

In [14]:
wind_data = pd.read_sql("select * from wind_data", con=engine)

In [15]:
wind_data = wind_data[['date', 'avg_daily_wind_speed', 'fastest_2_min_speed', 'fastest_5_min_speed']]
wind_data.head()

,date,avg_daily_wind_speed,fastest_2_min_speed,fastest_5_min_speed
0,2000-12-01,3.58,14.1,16.1
1,2000-12-02,2.91,8.1,8.9
2,2000-12-03,3.80,13.0,14.1
3,2000-12-04,2.91,8.9,10.1
4,2000-12-05,4.03,10.1,12.1


In [16]:
wind_data.dtypes

date                     object
avg_daily_wind_speed    float64
fastest_2_min_speed     float64
fastest_5_min_speed     float64
dtype: object

In [17]:
type(wind_data['date'][0])

str

In [18]:
wind_data['date'] = pd.to_datetime(wind_data['date'])

In [19]:
wind_data.dtypes

date                    datetime64[ns]
avg_daily_wind_speed           float64
fastest_2_min_speed            float64
fastest_5_min_speed            float64
dtype: object

## Merge them

In [20]:
grouped_acres.head()

,size
date,
2001-01-01,1.5
2001-01-02,0.4
2001-01-03,0.5
2001-01-04,9.8
2001-01-05,146.0


In [21]:
all_data = pd.merge(grouped_acres, wind_data, on="date")
all_data.head()

,date,size,avg_daily_wind_speed,fastest_2_min_speed,fastest_5_min_speed
0,2001-01-01,1.5,3.58,10.1,10.1
1,2001-01-02,0.4,2.91,10.1,10.1
2,2001-01-03,0.5,2.68,10.1,10.1
3,2001-01-04,9.8,3.58,8.9,10.1
4,2001-01-05,146.0,4.70,15.0,17.0


In [44]:
aqi.head()

,overall_aqi
date,
2001-01-01,153.0
2001-01-02,140.0
2001-01-03,140.0
2001-01-04,107.0
2001-01-05,124.0


In [45]:
all_data = pd.merge(all_data, aqi, on="date")

In [46]:
all_data.head()

,date,size,avg_daily_wind_speed,fastest_2_min_speed,fastest_5_min_speed,overall_aqi
0,2001-01-01,1.5,3.58,10.1,10.1,153.0
1,2001-01-02,0.4,2.91,10.1,10.1,140.0
2,2001-01-03,0.5,2.68,10.1,10.1,140.0
3,2001-01-04,9.8,3.58,8.9,10.1,107.0
4,2001-01-05,146.0,4.70,15.0,17.0,124.0


## Write to database

In [47]:
all_data.to_sql(name="final_daily_total_plus_wind", con=engine, if_exists="replace", index=True)